In [1]:
import pandas as pd
import pickle
import numpy as np
import tweepy 
import json
import pprint as pp
import sys
import time
import collections
import os

## Gather

Open two saved pandas DataFrames.  
<br>
The twitter_archive_common_df and the image_common_df have the same tweet_id(s).  
This was done in preparation for the __tweet_id__ mediated merge of these two DataFrames with the DataFrame Gathered using the tweepy.API. The following Gather process creates the 3rd required pandas DataFrame 
 

tweet_id() are common in both DataFrames
* image_common_df
* twitter_archive_common_df

In [2]:
# Retrieve the image data, instantiate the image_common_df pandas DataFrame
with open('image_common_df.pkl', 'rb') as f:
    image_common_df = pickle.load(f)
    
print('image_common_df.shape')
print(image_common_df.shape)

image_common_df.shape
(1982, 12)


In [3]:
# Retrieve the twitter_archive data, 
# instantiate the twitter_archive_common_df pandas DataFrame

with open('twitter_archive_common_df.pkl', 'rb') as f:
    twitter_archive_common_df = pickle.load(f)
    
print('twitter_archive_common_df.shape')
print(twitter_archive_common_df.shape)

twitter_archive_common_df.shape
(1982, 11)


In [4]:
# Create tweet_id driver list for tweepy.API
common_tweet_id_list = twitter_archive_common_df.tweet_id.tolist()
print('len(common_tweet_id_list) - {}'.format(len(common_tweet_id_list)))

# development / testing overrides
# common_tweet_id_list = [754011816964026368,666020888022790149]

len(common_tweet_id_list) - 1982


## Assess

Requirements compliance  
Data Analyst Nanodegree Program  
8\. Data Wrangling  
Project: Wrangle and Analyze Data  
4\. Twitter API

preparation for Tidy driven pandas Series tweet_id merge of 3 pandas DataFrames
* pandas tweet_id Series must have the
same content

## Clean

### Define

Capture missing, unmatched tweet_id(s)  
<br>
Requirements compliance  
* After querying each tweet ID, you will write its JSON data to the required tweet_json.txt file with each tweet's JSON data on its own line. 

### Code

In [5]:
# Real time tweepy.API gets 
# On the fly update tweet_json.txt
counter = 0
tweet_id_error_dictionary = {}
tweet_id_error_list = []

auth = tweepy.OAuthHandler('cZFGyPnfKa2ezCMqNMqSDYLEY', 'EMMU1n2TU415x15Q8UPD58P5hfBPAu1HeZWulF6xFEVjiRn40a')
auth.set_access_token('168897950-kHN9qymcxBKdxO6XtrncugxGqXxWibKecwpDGYCR',
                      'oHt0QgobyfWmQfUuHwPPBklQefHClsmzbiaROkUP8oYvc')

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

with open('tweet_json.txt', 'w') as file:
    for tweet_id in common_tweet_id_list:
        # print('tweet_id - {}'.format(tweet_id))
     
        attempts = 0
        counter +=1
        
        # mitigate sporious network errors
        # see Python try - except clauses below
        while attempts < 5: 
            attempts += 1 
            # populate tweet_json.txt
            try:
                tweepy_status = api.get_status(str(tweet_id), tweet_mode='extended')
                # print('tweepy_status - {}\n'.format(tweepy_status))
                #
                # tweepy_status - 
                # Status(_api=<tweepy.api.API object at 0x1136f3e80>,
                # _json={'created_at': 
                # 'Tue Aug 01 16:23:56 +0000 2017', 
                # 'id': 892420643555336193, 
                # 'id_str': '892420643555336193', 
                # 'full_text': "This is Phineas. ...

                # print('type(tweepy_status) - {}\n'.format(type(tweepy_status)))
                #       type(tweepy_status) - <class 'tweepy.models.Status'>

                json.dump(tweepy_status._json, file)
                file.write('\n')
                break
            # Capture missing tweet_id(s)
            except tweepy.TweepError as tweep_error:
                print('tweet_id - {}'.format(tweet_id))
                print('tweep_error - {}'.format(tweep_error))
                print('tweep_error.api_code - {}'.format(tweep_error.api_code))
                print('tweep_error.response - {}'.format(tweep_error.response))
                print('type(tweep_error.response) - {}\n'.format(type(tweep_error.response)))
            
                if tweep_error.response is not None: 
                    temp_list = tweep_error.response.text.split(',')
            
                    for my_string in temp_list:
                        # "message":"No status found with that ID."}]}
                        # print('my_string')
                        # print(my_string)
                        # print()
                        if 'message' in my_string and \
                        'No status found with that ID.' in my_string:
                            tweet_id_error_dictionary['tweet_id'] = tweet_id
                            tweet_id_error_dictionary['code'] = tweep_error.api_code
                            tweet_id_error_dictionary['message'] = my_string
            
                            tweet_id_error_list.append(tweet_id_error_dictionary)
                
                            tweet_id_error_dictionary = {}
                    break
                    
            # mitigate sporious network errors
            except Exception as err:
                print('Exception - sys.exc_info()') 
                print(sys.exc_info())
                print('tweet_id - {}'.format(tweet_id))
                print('counter - {}'.format(counter))
                print("err {}".format(err)) 
                print()

        if counter % 100 == 0:
            print('counter - {}'.format(counter))
            

counter - 100
counter - 200
counter - 300
counter - 400
counter - 500
counter - 600
tweet_id - 754011816964026368
tweep_error - [{'code': 144, 'message': 'No status found with that ID.'}]
tweep_error.api_code - 144
tweep_error.response - <Response [404]>
type(tweep_error.response) - <class 'requests.models.Response'>

counter - 700
counter - 800


Rate limit reached. Sleeping for: 708


counter - 900
tweet_id - 718613305783398402
tweep_error - Failed to send request: ('Connection aborted.', OSError("(60, 'ETIMEDOUT')",))
tweep_error.api_code - None
tweep_error.response - None
type(tweep_error.response) - <class 'NoneType'>

counter - 1000
counter - 1100
counter - 1200
counter - 1300
counter - 1400
counter - 1500
counter - 1600
counter - 1700


Rate limit reached. Sleeping for: 707


counter - 1800
tweet_id - 669353438988365824
tweep_error - Failed to send request: ('Connection aborted.', OSError("(60, 'ETIMEDOUT')",))
tweep_error.api_code - None
tweep_error.response - None
type(tweep_error.response) - <class 'NoneType'>

counter - 1900


## Test

In [6]:
print('\ntweet_id_error_list')
print(tweet_id_error_list)
print()

print('len(tweet_id_error_list)')
print(len(tweet_id_error_list))
print()



tweet_id_error_list
[{'tweet_id': '754011816964026368', 'code': 144, 'message': '"message":"No status found with that ID."}]}'}]

len(tweet_id_error_list)
1



Success  
Missing tweet_id(s) captured

took an hour plus, make sure tweet_json.txt is there

In [7]:
os.path.isfile('tweet_json.txt') \
and os.path.getsize('tweet_json.txt') > 0

True

Success  
Requirements compliance  
"you will write its JSON data to the required tweet_json.txt file"

In [8]:
# Save - took hour plus processing time
with open('tweet_id_error_list.pkl', 'wb') as f:
    pickle.dump(tweet_id_error_list, f)

# READ - make sure we can read it
with open('tweet_id_error_list.pkl', 'rb') as f:
    tweet_id_error_list = pickle.load(f)

print('tweet_id_error_list')
print(tweet_id_error_list)

tweet_id_error_list
[{'tweet_id': '754011816964026368', 'code': 144, 'message': '"message":"No status found with that ID."}]}'}]


Requirements compliance  
Data Analyst Nanodegree Program  
8\. Data Wrangling  
Project: Wrangle and Analyze Data  
4\. Twitter API  

You will then read this file, line by line, ... 

In [9]:
# populate tweet_data_list from the saved tweet_json.txt  
 
tweet_data_list = []
with open('tweet_json.txt', 'r') as json_file:
    for tweet_data in json_file:
        if tweet_data.strip() != '':
            tweet_data_list.append(tweet_data)
            
print('len(tweet_data_list) - {}\n'.format(len(tweet_data_list)))
     

len(tweet_data_list) - 1981



## Assess

Requirements compliance.   
Data Analyst Nanodegree Program  
8\. Data Wrangling  
Project: Wrangle and Analyze Data  
4\. Twitter API  

... to create a pandas DataFrame

## Clean

### Define

create pandas DataFrame from tweet_json.txt

### Code

In [10]:
retrieved_tweet_dictionary = {}

# instantiate new retrieved tweet pandas DataFrame 
retrieved_tweet_data_df = pd.DataFrame\
(columns = ['tweet_id','retweet_count','favorite_count'])

for tweet_data in tweet_data_list:
    # populate retrieved_tweet_dictionary with retrieved tweet_data
    retrieved_tweet_dictionary = json.loads(tweet_data)
    # print('retrieved_tweet_dictionary - {}'.format(retrieved_tweet_dictionary))

    # populate pandas DataFrame with retrieved tweet data 
    # tweet_id, retweet_count, favorite_count columns  
    
    retrieved_tweet_data_df = retrieved_tweet_data_df.append\
    ({'tweet_id':retrieved_tweet_dictionary['id'],\
    'retweet_count':retrieved_tweet_dictionary['retweet_count'],
    'favorite_count':retrieved_tweet_dictionary['favorite_count']}, \
     ignore_index = True)

### Test

In [11]:
print('retrieved_tweet_data_df.shape - {}\n'.\
      format(retrieved_tweet_data_df.shape))

retrieved_tweet_data_df.shape - (1981, 3)



Success  
Requirements compliance  
retrieved_tweet_data_df pandas DataFrame created from tweet_json.txt

## Assess

in preparation for Tidy driven tweet_id mediated pd.merge of 3 DataFrames  
pandas tweet_id Series must have the same content 

## Clean

### Define 

synchronize pandas tweet_id Series

### Code

In [12]:
tweet_id_to_be_removed_list = []

for error_tweet in tweet_id_error_list:
    tweet_id_to_be_removed_list.append(error_tweet.get('tweet_id'))
    
print('tweet_id_to_be_removed_list')
print(tweet_id_to_be_removed_list)
print()

print('twitter_archive_common_df.shape')
print(twitter_archive_common_df.shape)
print()

print('image_common_df.shape')
print(image_common_df.shape)
print()

print('retrieved_tweet_data_df.shape')
print(retrieved_tweet_data_df.shape)
print()

for tweet_id in tweet_id_to_be_removed_list:
    print('tweet_id - {}'.format(tweet_id))
    image_common_df = image_common_df[image_common_df.tweet_id != tweet_id]
    twitter_archive_common_df = \
    twitter_archive_common_df[twitter_archive_common_df.tweet_id != tweet_id]
    
print()
        
print('twitter_archive_common_df.shape')
print(twitter_archive_common_df.shape)
print()

print('image_common_df.shape')
print(image_common_df.shape)
print()

print('retrieved_tweet_data_df.shape')
print(retrieved_tweet_data_df.shape)
print()

twitter_archive_tweet_id_list = \
twitter_archive_common_df.tweet_id.tolist()

image_common_df_tweet_id_list =  image_common_df.tweet_id.tolist()
# hack to get the quotes put around each tweet_id
retrieved_tweet_id_as_str = [str(i) for i in retrieved_tweet_data_df.tweet_id.tolist()] 
print('len(retrieved_tweet_id_as_str) - {}'.format(len(retrieved_tweet_id_as_str)))


tweet_id_to_be_removed_list
['754011816964026368']

twitter_archive_common_df.shape
(1982, 11)

image_common_df.shape
(1982, 12)

retrieved_tweet_data_df.shape
(1981, 3)

tweet_id - 754011816964026368

twitter_archive_common_df.shape
(1981, 11)

image_common_df.shape
(1981, 12)

retrieved_tweet_data_df.shape
(1981, 3)

len(retrieved_tweet_id_as_str) - 1981


### Test

In [13]:
print(collections.Counter(twitter_archive_tweet_id_list) == \
      collections.Counter(image_common_df_tweet_id_list) ==\
      collections.Counter(retrieved_tweet_id_as_str))
print()

True



Success  
pandas Series tweet_id synchronized

In [14]:
# Save
with open('image_common_df.pkl', 'wb') as f:
    pickle.dump(image_common_df, f)

# READ
with open('image_common_df.pkl', 'rb') as f:
    image_common_df = pickle.load(f)

print('image_common_df.shape')
print(image_common_df.shape)

image_common_df.shape
(1981, 12)


In [15]:
# Save
with open('twitter_archive_common_df.pkl', 'wb') as f:
    pickle.dump(twitter_archive_common_df, f)

# READ
with open('twitter_archive_common_df.pkl', 'rb') as f:
    twitter_archive_common_df = pickle.load(f)

print('twitter_archive_common_df.shape')
print(twitter_archive_common_df.shape)


twitter_archive_common_df.shape
(1981, 11)


In [16]:
# Save
with open('retrieved_tweet_data_df.pkl', 'wb') as f:
    pickle.dump(retrieved_tweet_data_df, f)

# READ
with open('retrieved_tweet_data_df.pkl', 'rb') as f:
    retrieved_tweet_data_df = pickle.load(f)

print('retrieved_tweet_data_df.shape')
print(retrieved_tweet_data_df.shape)

retrieved_tweet_data_df.shape
(1981, 3)
